Задание 1

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете data/corpus_wsd_50k.txt

В качестве метрики близости вы должны попробовать два подхода:

Jaccard score на множествах слов (определений и контекста)

In [5]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
corpus_wsd = []
corpus = open('/content/corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [7]:
# находим все значения слова и все определения всех значений
def meanings(word):
    all_meanings = []
    word_to_check = wn.synonyms(word)

    for lists_of_synons in word_to_check:
        for synoms in lists_of_synons:
            for synset in wn.synsets(synoms):
                definiton = synset.definition()
                if definiton not in all_meanings:
                    all_meanings.append(definiton)

    return all_meanings

meanings('car')


['a motor vehicle with four wheels; usually propelled by an internal combustion engine',
 'travel in an automobile',
 'any mechanical or electrical device that transmits or modifies energy to perform or assist in the performance of human tasks',
 'an efficient person',
 'an intricate organization that accomplishes its goals efficiently',
 'a device for overcoming resistance at one point by applying force at some other point',
 'a group that controls the activities of a political party',
 'turn, shape, mold, or otherwise finish by machinery',
 'make by machinery',
 'a wheeled vehicle adapted to the rails of railroad',
 'a low flat-bottomed freight car with fixed sides but no roof',
 'long narrow flat-bottomed boat propelled by sculling; traditionally used on canals of Venice',
 'the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant',
 'where passengers ride up and down',
 'a conveyance for passengers or freight on a cable railway'

In [8]:
# для удобства сделал функцию, чтобы выводить значения и определния значений, вторым вывожу определение для wn
def definitions(word, wn_mean):

    definitions = meanings(word)
    wn_true = wn.lemma_from_key(wn_mean).synset().definition()

    return definitions, wn_true

definitions('car', 'long%3:00:02::')


(['a motor vehicle with four wheels; usually propelled by an internal combustion engine',
  'travel in an automobile',
  'any mechanical or electrical device that transmits or modifies energy to perform or assist in the performance of human tasks',
  'an efficient person',
  'an intricate organization that accomplishes its goals efficiently',
  'a device for overcoming resistance at one point by applying force at some other point',
  'a group that controls the activities of a political party',
  'turn, shape, mold, or otherwise finish by machinery',
  'make by machinery',
  'a wheeled vehicle adapted to the rails of railroad',
  'a low flat-bottomed freight car with fixed sides but no roof',
  'long narrow flat-bottomed boat propelled by sculling; traditionally used on canals of Venice',
  'the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant',
  'where passengers ride up and down',
  'a conveyance for passengers or freight on a

In [9]:
# считаем JACCARD для датасета
total_right = 0
total_sentences = 0

for sentences in corpus_wsd:

    sentence = ''

    for words in sentences:
        try:
            sentence = sentence + words[2] + ' '
        except:
            pass

    for words in sentences:

        if '' not in words:

            defs = definitions(words[2], words[0])
            sentence_without_target = sentence.replace(words[2], '_')

            for d in defs[0]:
                # подсчет JACCARD

                intersection1 = (set(d.split()) & set(sentence_without_target.split()))
                union1 = (set(d.split()) | set(sentence_without_target.split()))
                jaccard1 =  len(intersection1) / len(union1)

                intersection2 = (set(defs[1].split()) & set(sentence_without_target.split()))
                union2 = (set(defs[1].split()) | set(sentence_without_target.split()))
                jaccard2 =  len(intersection2) / len(union2)

                if sentence_without_target:
                  total_sentences += 1
                if jaccard1 != 0.0 and jaccard1 == jaccard2:
                    total_right += 1

                # print(f" \n target_word: {words[2]} \n target_sent: {sentence_without_target}  \n definition_synset: {d}  \n definition_wn: {defs[1]} \n jaccard_synset: {jaccard1} \n jaccard_wn: {jaccard2}")

# total sentences включают в себя все итеррации по всем существующим значениям для слова.
# Я решил считать так, чтобы понять сколько всего мы проходим и отбрасываем для нахождения правильного варианта
# Предполагаю, что это не совсем то, что надо было найти, но так мы поймем каков шанс на то, что среди всех значений будет подходящее, это может быть полезно, наверное...)

result = round(total_right/total_sentences * 100, 2)
print(f"total_right: {total_right}, total_sentences: {total_sentences}, accuracy для всех определений всех значений: {result}%")



total_right: 365391, total_sentences: 18678216, accuracy для всех определений всех значений: 1.96%


Задание 1

Cosine distance на эмбедингах sentence_transformers

In [ ]:
!python -m pip install torch torchvision torchaudio
!python -m pip install sentence_transformers transformers accelerate -U

In [11]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
# embedding model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

In [ ]:
# считаем косинус для датасета
total_right = 0
total_sentences = 0

for sentences in corpus_wsd[0]:

    sentence = ''

    for words in sentences:
        try:
            sentence = sentence + words[2] + ' '
        except:
            pass

    for words in sentences:

        if '' not in words:

            defs = definitions(words[2], words[0])
            # print(words[2], defs)
            sentence_without_target = sentence.replace(words[2], '_')
            context_emb = embed(sentence_without_target)

        for d in defs[0]:

            # подсчет косинус
            d_syn = embed(d)
            d_wn = embed(defs[1])


            distance_d_syn = cosine_distances(context_emb.reshape(1, -1), d_syn.reshape(1, -1))
            distance_d_wn = cosine_distances(context_emb.reshape(1, -1), d_wn.reshape(1, -1))
            sim = cosine_similarity(context_emb.reshape(1, -1), [d_syn, d_wn])

            if cosine_similarity:
                total_sentences += 1
            if sim[0][0] == sim[0][1]:
                total_right += 1

            # print(f"distance_d_syn: {distance_d_syn} \n distance_d_wn: {distance_d_wn} \n sim: {sim}")
result = round(total_right/total_sentences * 100, 2)

# Из-за того, что определений значений очень много выполнение невероятно долгое..
# Подскажите, пожалуйста, может быть надо переделать алгоритм и все же проходить не по всем определниям всех слов?

print(f"total_right: {total_right}, total_sentences: {total_sentences}, accuracy для всех определений всех значений: {result}%")